In [2]:
import argparse
from pathlib import Path
import sys
from datetime import datetime
from pandas import date_range

sys.path.append('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/python')
from eddyproconfigeditor import EddyproConfigEditor, compare_configs

wd = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m_heating_investigate')
arcc_wd = Path('/project/eddycovworkflow/afox18/Platinum-EddyPro7/workflows/BB-NF_17m_heating_investigate')
# arcc_wd = wd
data_dir = Path('/gscratch/afox18/eddycovworkflow/InputData/Chimney')
# data_dir = Path('/Users/alex/Documents/Data/Platinum_EC/InputData/Chimney')


for method in ['nearest', 'linear']:
    for sample in range(1, 7):  
        # MODIFY THE OUTDIR TO REFLECT THE CALIBRATION
        out_dir = Path(f'/gscratch/afox18/eddycovworkflow/ExpectedOutputs/Chimney/BB-NF/17m_heating')
        # out_dir = Path('/Users/alex/Documents/Data/Platinum_EC/ExpectedOutputs/Chimney/17m')
        template = EddyproConfigEditor(wd / 'ini/BB-NF_17m_template.eddypro')
        start, end = '2019-01-01 00:00', '2023-12-31 23:30'
        template.Proj.set_project_name('BB-NF_17m_heating')
        template.Proj.set_metadata(arcc_wd / 'ini/BB-NF_17m.metadata')
        template.Proj.set_biomet(mode='dir', path=data_dir / 'biomet/BB-NF/EC/17m/EddyPro_Biomet', extension='dat', subfolders=False)
        template.Basic.set_raw_data(path=data_dir / f'raw/BB-NF/EC/17m/Calibrated-{method}-{sample}', fmt='yyyy_mm_dd_HHMM.dat', subfolders=False)
        template.Basic.set_out_path(out_dir)
        template.Basic.set_project_date_range(start=start, end=end)
        template.Basic.set_missing_samples_allowance(pct=10)
        template.Basic.set_flux_averaging_interval(minutes=30)
        template.Basic.set_north_reference(method='mag')
        template.Adv.Proc.set_wind_speed_measurement_offsets(0, 0, 0)
        # MODIFY THE PF FILE TO POINT TO THE CORRECT ONE FOR THIS YEAR...OR SOMETHING
        template.Adv.Proc.set_axis_rotations_for_tilt_correction(
            method='planar_fit',
            
            pf_file=arcc_wd / 'eddypro_template-202101010000_planar_fit.txt'
        )
        template.Adv.Proc.set_turbulent_fluctuations(detrend_method='block')
        template.Adv.Proc.set_timelag_compensations(method='covariance_maximization_with_default')
        template.Adv.Spec.set_calculation(binned_cosp_dir=Path('/gscratch/afox18/eddycovworkflow/ExpectedOutputs/Chimney/BB-NF/17m/Template_Parallel/eddypro_binned_cospectra'))
        template.Adv.Out.set_spectral_output(
            binned_spectra=True,
            binned_ogives=True,
            ensemble_spectra=True,
            full_spectra='none',
            full_cospectra='none'
        )
        template.Adv.Out.set_chain_of_custody(
            unprocessed='stats',
            despiked='stats',
            timelag_corrected='stats',
            variables=['u', 'v', 'w', 'ts', 'co2', 'h2o', ]
        )
        # worker_windows = [datetime(y, 1, 1, 0, 0, 0) for y in range(2019, 2025)]
        worker_windows = [datetime.strptime(d, '%Y-%m-%d %H:%M') for d in date_range('2018-12-01', '2024-01-01', freq='30d').strftime('%Y-%m-%d %H:%M')]
        # worker_windows
        # worker_windows = [datetime(2022, 12, 1, 0, 0), datetime(2022, 12, 31, 0, 0), datetime(2023, 1, 31, 0, 0)]
        # template.to_eddypro(ini_file=wd / 'ini/BB-NF_17m_template.eddypro')

        # use custom self-heating
        template.Basic.set_output_id(output_id=f'selfheating-custom-multi')
        template.Adv.Proc.set_compensation_of_density_fluctuations(
            burba_method='multiple',
            day_bot=[3.6587, -0.1026, 0.0021, -0.5765],
            day_top=[0.9806, -0.0457, 0.0014, -0.2619],
            day_spar=[1.1756, -0.047, 0.0014, -0.2856],
            night_bot=[2.5341, -0.1444, 0, -0.4731],
            night_top=[0.1562, -0.034, 0, -0.082],
            night_spar=[0.4645, -0.0398, 0, -0.1344]
        )

        template.to_eddypro_parallel(
            ini_dir=wd / 'ini/BB-NF_17m_Parallel',
            out_path=out_dir,
            file_duration=1440,
            worker_windows=worker_windows
        )

        # use default self-heating with single variable
        template.Basic.set_output_id(output_id=f'selfheating-default-single')
        template.Adv.Proc.set_compensation_of_density_fluctuations(
            burba_method='simple',
            set_all='revert'
        )

        template.to_eddypro_parallel(
            ini_dir=wd / 'ini/BB-NF_17m_Parallel',
            out_path=out_dir,
            file_duration=1440,
            worker_windows=worker_windows
        )

        # use custom self-heating with single variable
        template.Basic.set_output_id(output_id=f'selfheating-custom-single')
        template.Adv.Proc.set_compensation_of_density_fluctuations(
            burba_method='simple',
            day_bot=[0.8578, 2.1008],
            day_top=[0.9413, 0.488],
            day_spar=[0.9367, 0.5453],
            night_bot=[0.7968, 0.4813],
            night_top=[0.9558, -0.1998],
            night_spar=[0.9435, -0.1185],
        )

        template.to_eddypro_parallel(
            ini_dir=wd / 'ini/BB-NF_17m_Parallel',
            out_path=out_dir,
            file_duration=1440,
            worker_windows=worker_windows
        )

In [79]:
len(worker_windows) - 1

265